In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import re
import scrapbook as sb

from bmondata import Server
from dateutil.relativedelta import relativedelta

import bmondata

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
building_id = 2
server_web_address = 'https://bms.ahfc.us'

In [ ]:
sb.glue('title', 'IAQ')
sb.glue('sort_order', 3)

server = Server(server_web_address)

building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']

# Get list of CO2 sensors
co2_sensors = bmondata.csnl_to_list(server.buildings(building_id)[0]['co2_sensors'])

if len(co2_sensors) == 0:
    error_message = 'This building does not appear to have any data on CO2 concentrations. Either there is no data source or it has not been properly configured.'
    raise RuntimeError(error_message)   

diverging_hue_list = ['#a6cee3',
                      '#1f78b4',
                      '#b2df8a',
                      '#33a02c',
                      '#fb9a99',
                      '#e31a1c',
                     '#fdbf6f']

title_md = '''# Building Ventilation for {} building'''
title_md = title_md.format(current_building_name)

###########################################################

Markdown(title_md)

In [ ]:
if server.sensor_readings(co2_sensors,start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                          end_ts = datetime.datetime.now(),averaging='15min').empty:
    if server.sensor_readings(co2_sensors,start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                              end_ts = datetime.datetime.now(),averaging='1H').empty:
        co2_current_week = server.sensor_readings(co2_sensors,start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                                  end_ts = datetime.datetime.now(),averaging='2H')
    else:
        co2_current_week = server.sensor_readings(co2_sensors,start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                                  end_ts = datetime.datetime.now(),averaging='1H')
else:
    # Get CO2 sensor data for the current week
    co2_current_week = server.sensor_readings(co2_sensors,start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                              end_ts = datetime.datetime.now(),averaging='15min')

# Get hourly CO2 data for the current year
co2_this_year = server.sensor_readings(co2_sensors, 
                                      start_ts = (datetime.datetime.now() - relativedelta(years=1)),
                                      end_ts = datetime.datetime.now(),
                                      averaging='1H')

# Get hourly CO2 data for the current year
this_week_outdoor_temps = server.sensor_readings(building_df[0]['outdoor_temp'], 
                                      start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                      end_ts = datetime.datetime.now(),
                                      averaging='1H')

mean_outdoor_temp = this_week_outdoor_temps.mean()[0]

if co2_current_week.min().min() > 400:
    min_value = 400 
else:
    min_value = co2_current_week.min().min()
    
if co2_current_week.max().max() > 1000:
    max_value = co2_current_week.max().max()
else: 
    max_value = 1000

# Get schedule and timezone from the API data
building_schedule = building_df[0]['schedule']
building_timezone = building_df[0]['timezone']

# Create a schedule object using Ian's library
schedule_object = bmondata.Schedule(building_schedule, building_timezone)

if schedule_object is None:
    shape_list = []
else:
    # Set the start and end times of the graph
    graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
    graph_end_date = datetime.datetime.now()

    # Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
    list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

    # Loop through the list of occupied timestamps and convert them to datetimes
    start_time_list = []
    end_time_list = []

    for i in np.arange(0, len(list_of_occupied_timestamps)):
        start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
        start_time_list.append(start_datetime)
        end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
        end_time_list.append(end_datetime)

    # Create variables to indicate the first occupied start time and the last occupied end time
    occ_start = start_time_list[0]
    occ_end = end_time_list[-1]
    occ_day_list = np.arange(occ_start.weekday(), occ_end.weekday()+1)
    
    shape_list = []
    shape_dict = {'type':'rect',
                  'fillcolor':'#bdbdbd',
                  'opacity':0.35,
                  'line': {'width':1},
                  'layer':'below',
                  'y0':min_value,
                  'y1':max_value
                 }
    
    if graph_start_date < start_time_list[0]:
        shape_dict.update(x0=graph_start_date)
        shape_dict.update(x1=start_time_list[0])
        shape_list.append(shape_dict.copy())
        
    for i in np.arange(len(start_time_list)):
        if start_time_list[-1] != start_time_list[i]:
            shape_dict.update(x0=end_time_list[i])
            shape_dict.update(x1=start_time_list[i+1])
            shape_list.append(shape_dict.copy())
            
    if graph_end_date > end_time_list[-1]:
        shape_dict.update(x0=end_time_list[-1])
        shape_dict.update(x1=graph_end_date)
        shape_list.append(shape_dict.copy()) 

###########################################################################################

counter = 1

data = []
for co2_sensor in co2_sensors:
    co2_sensor = go.Scatter(x=co2_current_week.index,
                             y=co2_current_week[co2_sensor],
                             name="CO2 sensor " + co2_sensor,
                             line=dict(color=diverging_hue_list[counter % len(diverging_hue_list)]))
    counter += 1
    data.append(co2_sensor)

data.append(go.Scatter(x=[co2_current_week.index.min(), co2_current_week.index.max()], 
                       y=[400, 400],
                       mode="lines",
                       name='Outdoor CO2 Concentration',
                       line=dict(dash='dashdot')
                      ))

data.append(go.Scatter(x=[co2_current_week.index.min(), co2_current_week.index.max()], 
                       y=[800, 800],
                       mode="lines",
                       name='Acceptable IAQ limit',
                       line=dict(dash='dashdot')
                      ))

layout = dict(title='Current Week CO2 Levels ',
              xaxis=dict(title='Date and Time'),
              yaxis=dict(title='Indoor CO2 Levels (ppm)'),
              shapes = shape_list)

fig = go.Figure(dict(data=data, layout=layout))
fig.show()

In [ ]:
list_of_md_results = []

for co2_sensor in co2_sensors:
    max = co2_current_week[co2_sensor].max()
    min = co2_current_week[co2_sensor].min()
    if max - min < 300 and mean_outdoor_temp < 45:
        md_results = '''#### Data for sensor {} suggests that this area may be over-ventilated; only if COVID-19 infection rates are under control, reducing the ventilation rate would potentially reduce energy costs while still maintaining acceptable indoor air quality.'''
    elif max - min >= 450:
        md_results = '''#### Data for sensor {} suggests that this area may be under-ventilated; 
        if the area is occupied, ventilation should be increased to ensure adequate indoor air quality.'''
    else:
        md_results = ''''''
        
    md_result = md_results.format(co2_sensor) 

    list_of_md_results.append(md_result)


###################################################################################

    
for md_result_x in list_of_md_results: 
    display(Markdown(md_result_x))

counter = 1

data = []
for co2_sensor in co2_sensors:
    co2_sensor = go.Scatter(x=co2_this_year.index,
                             y=co2_this_year[co2_sensor],
                             name="CO2 sensor: " + co2_sensor,
                             line=dict(color=diverging_hue_list[counter % len(diverging_hue_list)]))
    counter += 1
    data.append(co2_sensor)
    
data.append(go.Scatter(x=[co2_this_year.index.min(), co2_this_year.index.max()], 
                       y=[400, 400],
                       mode="lines",
                       name='Outdoor CO2 Concentration',
                       line=dict(dash='dashdot')
                      ))

data.append(go.Scatter(x=[co2_this_year.index.min(), co2_this_year.index.max()], 
                       y=[800, 800],
                       mode="lines",
                       name='Acceptable IAQ limit',
                       line=dict(dash='dashdot')
                      ))

layout = dict(title='Current Year CO2 Levels ',
              xaxis=dict(title='Date and Time'),
              yaxis=dict(title='Indoor CO2 Levels (ppm)'),
             )


fig = go.Figure(dict(data=data, layout=layout))
fig.show()